In [1]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel

In [2]:
spark.stop()

NameError: name 'spark' is not defined

In [3]:
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        config("spark.local.dir","shuffle/").\
        getOrCreate()

24/07/19 08:41:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/19 08:41:32 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [6]:
spark.conf.get('spark.local.dir')

'shuffle/'

In [7]:
spark.sparkContext.getConf().getAll()

[('spark.local.dir', 'shuffle/'),
 ('spark.executor.memory', '512m'),
 ('spark.app.id', 'app-20240719084133-0000'),
 ('spark.driver.host', 'c0168aacce39'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.port', '36027'),
 ('spark.master', 'spark://spark-master:7077'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'pyspark-notebook'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [8]:
dim_customer = spark.read\
                    .option("inferSchema", "true")\
                    .option("header", "true")\
                    .csv("dim_customer.csv")

In [9]:
dim_customer.schema

StructType(List(StructField(CustomerKey,IntegerType,true),StructField(Prefix,StringType,true),StructField(FirstName,StringType,true),StructField(LastName,StringType,true),StructField(BirthDate,StringType,true),StructField(MaritalStatus,StringType,true),StructField(Gender,StringType,true),StructField(EmailAddress,StringType,true),StructField(AnnualIncome,StringType,true),StructField(TotalChildren,IntegerType,true),StructField(EducationLevel,StringType,true),StructField(Occupation,StringType,true),StructField(HomeOwner,StringType,true)))

In [10]:
dim_customer.createOrReplaceTempView("dim_Customer")

In [11]:
spark.sql("""SELECT COUNT(*) as count_line FROM dim_Customer""").show()

+----------+
|count_line|
+----------+
|     18148|
+----------+



In [68]:
check = spark.sql("""SELECT * FROM dim_Customer where CustomerKey>=23500""")
check.show()

+-----------+------+----------+---------+----------+-------------+------+--------------------+------------+-------------+---------------+------------+---------+
|CustomerKey|Prefix| FirstName| LastName| BirthDate|MaritalStatus|Gender|        EmailAddress|AnnualIncome|TotalChildren| EducationLevel|  Occupation|HomeOwner|
+-----------+------+----------+---------+----------+-------------+------+--------------------+------------+-------------+---------------+------------+---------+
|      23500|  MRS.|   ADRIANA|   GARCIA|  6/6/1936|            M|     F|adriana16@adventu...|    $80,000 |            4|Graduate Degree|  Management|        Y|
|      23501|  MRS.|  ISABELLE|    PERRY|  7/2/1936|            M|     F|isabelle7@adventu...|    $90,000 |            4|      Bachelors|  Management|        Y|
|      23502|   MR.|     LOGAN|    LOPEZ|  9/7/1936|            M|     M|logan46@adventure...|   $110,000 |            2|Graduate Degree|  Management|        Y|
|      23503|   MR.|    DALTON|   

In [12]:
dim_customer.show()

+-----------+------+---------+--------+---------+-------------+------+--------------------+------------+-------------+---------------+--------------+---------+
|CustomerKey|Prefix|FirstName|LastName|BirthDate|MaritalStatus|Gender|        EmailAddress|AnnualIncome|TotalChildren| EducationLevel|    Occupation|HomeOwner|
+-----------+------+---------+--------+---------+-------------+------+--------------------+------------+-------------+---------------+--------------+---------+
|      11000|   MR.|      JON|    YANG| 4/8/1966|            M|     M|jon24@adventure-w...|    $90,000 |            2|      Bachelors|  Professional|        Y|
|      11001|   MR.|   EUGENE|   HUANG|5/14/1965|            S|     M|eugene10@adventur...|    $60,000 |            3|      Bachelors|  Professional|        N|
|      11002|   MR.|    RUBEN|  TORRES|8/12/1965|            M|     M|ruben35@adventure...|    $60,000 |            3|      Bachelors|  Professional|        Y|
|      11003|   MS.|  CHRISTY|     ZHU|2

In [13]:
fact_sales = spark.read\
                .option("inferSchema", "true")\
                .option("header", "true")\
                .csv("fact_sales.csv")

In [14]:
fact_sales.show()

+---------+----------+-----------+----------+-----------+------------+-------------+-------------+
|OrderDate| StockDate|OrderNumber|ProductKey|CustomerKey|TerritoryKey|OrderLineItem|OrderQuantity|
+---------+----------+-----------+----------+-----------+------------+-------------+-------------+
| 1/1/2016|10/17/2002|    SO48797|       385|      14335|           1|            1|            1|
| 1/1/2016| 9/30/2002|    SO48802|       383|      24923|           9|            1|            1|
| 1/1/2016|11/29/2002|    SO48801|       326|      15493|           1|            1|            1|
| 1/1/2016|11/16/2002|    SO48799|       352|      26708|           4|            1|            1|
| 1/1/2016|12/16/2002|    SO48798|       369|      23332|           9|            1|            1|
| 1/1/2016| 12/2/2002|    SO48800|       342|      15491|           5|            1|            1|
| 1/1/2016|10/19/2002|    SO48795|       375|      16538|           8|            1|            1|
| 1/1/2016

In [17]:
fact_sales.persist(StorageLevel.DISK_ONLY)

24/07/19 08:42:50 WARN CacheManager: Asked to cache already cached data.


DataFrame[OrderDate: string, StockDate: string, OrderNumber: string, ProductKey: int, CustomerKey: int, TerritoryKey: int, OrderLineItem: int, OrderQuantity: int]

In [18]:
fact_sales.rdd.getNumPartitions()

1

In [19]:
fact_sales.show()

+---------+----------+-----------+----------+-----------+------------+-------------+-------------+
|OrderDate| StockDate|OrderNumber|ProductKey|CustomerKey|TerritoryKey|OrderLineItem|OrderQuantity|
+---------+----------+-----------+----------+-----------+------------+-------------+-------------+
| 1/1/2016|10/17/2002|    SO48797|       385|      14335|           1|            1|            1|
| 1/1/2016| 9/30/2002|    SO48802|       383|      24923|           9|            1|            1|
| 1/1/2016|11/29/2002|    SO48801|       326|      15493|           1|            1|            1|
| 1/1/2016|11/16/2002|    SO48799|       352|      26708|           4|            1|            1|
| 1/1/2016|12/16/2002|    SO48798|       369|      23332|           9|            1|            1|
| 1/1/2016| 12/2/2002|    SO48800|       342|      15491|           5|            1|            1|
| 1/1/2016|10/19/2002|    SO48795|       375|      16538|           8|            1|            1|
| 1/1/2016